# Group 2 Electricity

### Imports

In [25]:
import warnings
import sys
import os.path
import math
sys.path.insert(0, os.path.abspath('../../../../functions'))
from knmi_weather import knmi_weather as knmi # eda tools
from sp_ql import Engine
import pandas as pd
import statistics
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
from math import ceil
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error
import joblib
import seaborn as sns
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold

### Defs

In [26]:
def calc_soc_pos_on_plot(df_compare_monthly, soc):
    sm = list(df_compare_monthly.index < soc).index(False) # first False is start month
    return sm - 0.5 # place of dashed line on plot in 0-1 x coord range

### Read Inputs

In [7]:
project = 'c2'
engine=Engine(project=project, isAsync=False)
bms_registers = engine.list().filter(unit='CELCIUS').all()
build_names = bms_registers['keyname_01'].unique()
build_names = build_names[~pd.isna(build_names)]
eans=pd.read_csv(f'/Users/Matt/Desktop/MSc/Thesis/Spectral/brighter_eda/inputs/eans/{project.lower()}_eans.csv')
#extraction of unique keyname_01 and EAN for electricity
electr_eans=eans[eans['Contract Type']=='Electricity']
electr_eans=electr_eans[['keyname_01', 'EAN']].dropna().set_index('keyname_01').squeeze()

#adds missing buildings
for building in build_names:
    if building not in electr_eans.index:
        electr_eans[building]=np.nan

In [1]:
del electr_eans['P******']
del electr_eans['CA*****']
del electr_eans['CR*****']
del electr_eans['A******']
del electr_eans['S******']
del electr_eans['H******']
del electr_eans['O******']
del electr_eans['O6*****']
del electr_eans['O1*****']
del electr_eans['T******']
del electr_eans['TU*****'] #not enough training data

In [658]:
#removes two buildings from build_names which have both no electricity and gas EAN data
build_names=np.delete(build_names, 10, axis=None)

In [659]:
# determine building location
cities = eans[['keyname_01', 'City']].dropna()
cities = cities.drop_duplicates().set_index('keyname_01').squeeze()
cities['P*****']='Amsterdam' #manually adding: *****
cities['R*****']='Amsterdam' #change later
cities['R*****']='Amsterdam' #change later
cities['R*****']='Amsterdam' #change later
cities['R*****']='Amsterdam' #change later
# read and preprocess weather data
knmi_station_lookup = pd.read_csv(r'../../../../inputs/knmi/knmi_uurgegevens/city_knmi_station_lookup.csv', index_col=0, squeeze=True)
#KeyError: "['*****', '*****', '*****', '*****', '*****'] not in index"
t = cities.to_frame()
t = t[~t.index.duplicated(keep='first')]
cities=t.squeeze()

In [13]:
# read in start of control dates, just reads the file
socs = pd.read_csv(r'../../../../inputs/start_of_control/start_of_control_dates.csv')
socs = socs[socs['project']==project]
socs = socs.set_index('keyname_01')['soc_date'].squeeze()
socs = pd.to_datetime(socs)

In [20]:
data_periods = pd.DataFrame()
data_periods['start'] = socs[electr_eans.index] - dt.timedelta(days=365)
data_periods['end'] = socs[electr_eans.index]
data_periods=data_periods.drop_duplicates(subset=['start'])
data_periods.loc['G*****', 'start']

Timestamp('2018-03-15 00:00:00')

### Pulling data

In [ ]:
engine = Engine(project=project, isAsync=False)
time_res = 'h'
electr_use = pd.DataFrame(index=pd.date_range(start=data_periods['start'].min(),
                                              end=data_periods['end'].max(),
                                              freq=time_res))

# pulls data from sp-ql
electr_builds = ['G*****', 'H4*****', 'K******', 'V******', 'H8*****', 'W******']
for building in [b for b in build_names if b in electr_builds]:
#skips buildings with no electricity EAN data
    start = data_periods.loc[building, 'start']
    end = data_periods.loc[building, 'end']
    electr_ean = electr_eans[building]
    if isinstance(electr_ean, str):
        electr = engine.select(electr_ean).group_by(time_res).mean().after(start).before(end).all() #the result is now kW power in hourly resolution, which is numerically identical to the hourly kWh value
        if len(electr) > 0:
            electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
            electr_use = electr_use.join(electr)
        #else:
            #print(fr'{building} - electricity EAN:{electr_eans[building]} not found')
    else:
        for ean in electr_ean:
            electr = engine.select(ean).group_by(time_res).mean().after(start).before(end).all()
            if len(electr) > 0:
                electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
                electr_use = electr_use.join(electr)
            #else:
                #print(fr'{building} - electricity EAN:{ean} not found')
electr_use = electr_use.resample('d').sum() #sums the kWh to daily readings
filter_col = [col for col in electr_use if col.startswith('W***')] #filters the buildings that start with input name
electr_use[filter_col]=electr_use[filter_col].fillna(0) #adds 0's for NaN's for summation of electricity usage
electr_use['W******']=electr_use[filter_col].sum(axis=1) #summation of all buildings with name filter
electr_use['W******']=electr_use['W*****'].replace({'0':np.nan, 0:np.nan}) #replacing the 0's back for NaN's
electr_use=electr_use.drop(['W*****', 'W*****'], axis=1)
electr_use.rename(columns = {'G*****':'G*****', 'H*****':'H4*****', 'K*****':'K*****', 
                                     'V*****':'V*****', 'H8****':'H8****', 'T*****':'T*****',
                             'W*****':'W*****'}, inplace = True)

In [663]:
weather = {}
for station in knmi_station_lookup.unique():
    knmi_weather_file_1 = f'../../../../inputs/knmi/knmi_uurgegevens/uurgeg_{station}_2011-2020.txt'
    knmi_weather_file_2 = f'../../../../inputs/knmi/knmi_uurgegevens/uurgeg_{station}_2021-2030.txt'
    knmi_weather = pd.read_csv(knmi_weather_file_1,
                               skiprows=31,
                               skipinitialspace=True,
                               usecols=list(range(1, 25)))
    knmi_weather = knmi_weather.append(pd.read_csv(knmi_weather_file_2,
                                                   skiprows=31,
                                                   skipinitialspace=True,
                                                   usecols=list(range(1, 25))))
    # making proper index for KNMI weather data
    knmi_weather.index = knmi.convert_knmi_dates(date=knmi_weather['YYYYMMDD'],
                                                 hour=knmi_weather['HH'])
    # clean up values
    knmi_weather = knmi.cleanup_knmi_units(knmi_weather=knmi_weather)
    weather[station] = knmi_weather

In [664]:
#was used for forecasting later on
#knmi_weather=weather[275]
#features = pd.DataFrame()
#features['SQ'] = knmi_weather['SQ'].resample('d').mean()
#features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
#features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
#features.to_csv('features_275.csv')

In [ ]:
## Determine training and prediction intervals
train_periods = pd.DataFrame()
train_periods['end'] = socs[electr_use.columns]
train_periods['start'] = socs[electr_use.columns] - dt.timedelta(days=365) #more training data=better
train_periods = train_periods[~train_periods.index.duplicated(keep='first')]
train_periods

In [666]:
electr_use.to_csv(r'c2_electr/data/electr_use_train.csv')

In [667]:
# some buildings have small gaps in gas data. We can interpolate over these
electr_use_train = pd.read_csv(r'c2_electr/data/electr_use_train.csv', index_col=0,
                            parse_dates=True)

In [668]:
for building in electr_use.columns:
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    electr_use_train_build = electr_use_train.loc[start_train:end_train, building]
    #setting first element to 0 if nan. To aid interpolation.
    if electr_use_train_build.iloc[[0]].isna()[0]:
        electr_use_train_build.iloc[0] = 0
    electr_use_train_build = electr_use_train_build.interpolate(limit=10) # filling max 10 consecutive days of missing data?
    electr_use_train.loc[start_train:end_train, building] = electr_use_train_build

### Random Forest

In [ ]:
for building in electr_use.columns:
    print(fr'Training building {building}, {electr_use.columns.get_loc(building) + 1}/{len(electr_use.columns)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    features = pd.DataFrame()

    features['SQ'] = knmi_weather['SQ'].resample('d').mean()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek
    
    electr_use_train_build = electr_use[building]
    
    # X: training features, y: training target
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = electr_use_train_build[(electr_use_train_build.index >= start_train) &
                            (electr_use_train_build.index < end_train)]
    #
    # checking if there is any nan in the data
    if X.isna().mean().mean() + y.isna().mean() > 0:
        raise ValueError('NaN in training data:', building)
    #
    # checking on dependency of taget on features to get a feel
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    #
    # make train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    #
    # grid-search for hyper parameter tuning
    param_grid = {'max_depth': [2, 5, 10, 20, 50, 100],
                  'min_samples_leaf': [1, 5, 10],
                  'n_estimators': [500]}
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                              cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    #
    # save the trained RF
    joblib.dump(grid_search.best_estimator_, fr'c2_electr/data/trained_models/rf/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_electr/data/trained_models/rf/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_electr/data/trained_models/rf/y_test_{building}.joblib')

### Gradient Boosting Regressor

In [ ]:
for building in electr_use.columns:
    print(fr'Training building {building}, {electr_use.columns.get_loc(building) + 1}/{len(electr_use.columns)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    features = pd.DataFrame()

    features['SQ'] = knmi_weather['SQ'].resample('d').mean()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek
    
    electr_use_train_build = electr_use[building]
    
    X = features[(features.index >= start_train) & (features.index < end_train)]
    
    y = electr_use_train_build[(electr_use_train_build.index >= start_train) &
                            (electr_use_train_build.index < end_train)]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


    param_grid = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10],
    }
    
    rf = GradientBoostingRegressor()
    
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                        cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    
    joblib.dump(grid_search.best_estimator_, fr'c2_electr/data/trained_models/gb/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_electr/data/trained_models/gb/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_electr/data/trained_models/gb/y_test_{building}.joblib')

### XGBoosting

In [ ]:
for building in electr_use.columns:
    print(fr'Training building {building}, {electr_use.columns.get_loc(building) + 1}/{len(electr_use.columns)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    features = pd.DataFrame()
    
    features['SQ'] = knmi_weather['SQ'].resample('d').mean()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek

    electr_use_train_build = electr_use[building]
    X = features[(features.index >= start_train) & (features.index < end_train)] #KNMI data (used for training) before SOC (10-12months) up until SOC
    y = electr_use_train_build[(electr_use_train_build.index >= start_train) &
                            (electr_use_train_build.index < end_train)]
    if X.isna().mean().mean() + y.isna().mean() > 0:
        raise ValueError('NaN in training data:', building)
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    param_grid={
                "learning_rate": (0.05, 0.10, 0.15, 0.2),
                "max_depth": [3, 4, 5, 6, 8],
                "min_child_weight": [1, 3, 5, 7]
                }

    rf = XGBRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                              cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)

    grid_search.fit(X_train, y_train)

    joblib.dump(grid_search.best_estimator_, fr'c2_electr/data/trained_models/xg/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_electr/data/trained_models/xg/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_electr/data/trained_models/xg/y_test_{building}.joblib')

In [ ]:
importance={}
for building in electr_use.columns:
    models = os.listdir('c2_electr/data/trained_models/xg') #change for RF, GB, XG
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_electr/data/trained_models/xg/trained_{building}.joblib') #change for RF, GB, XG
        importance[building]=grid_search.feature_importances_
importance_df=pd.concat({k: pd.DataFrame(v) for k, v in importance.items()}, axis=1)
importance_df['mean'] = importance_df.mean(axis=1)
importance_df.insert(0, 'FEATURE', features.columns)
importance_df=importance_df.set_index(importance_df.columns[0])
importance_df=importance_df.sort_values(by=['mean'],ascending=False)
importance_df

### Metrics - single

In [746]:
# reading in saved trained RF models and calculating mae for the test set
mae_range_percent = pd.Series()
for building in electr_use.columns:
    models = os.listdir('c2_electr/data/trained_models/rf') #change for RF, GB, XG
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_electr/data/trained_models/rf/trained_{building}.joblib') #change for RF, GB, XG
        X_test = joblib.load(fr'c2_electr/data/trained_models/rf/X_test_{building}.joblib') #change for RF, GB, XG
        y_test = joblib.load(fr'c2_electr/data/trained_models/rf/y_test_{building}.joblib') #change for RF, GB, XG
    #
    # grid search already contains the re-trained model with the best parameters
    y_test_pred = grid_search.predict(X_test)
    #
    # calculating MAE and plotting test and pred_test
    mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    # mae in percent of range
    mae_range_percent[building] = mae / y_test.max() * 100
    
    electr_use_train_build = electr_use[building]

In [747]:
statistics.mean(mae_range_percent)

5.763332972984917

### Metrics together - MAE

In [748]:
metric_com_mae = pd.DataFrame(index=electr_use.columns)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    mae_range_percent = pd.Series()
    for building in electr_use.columns:
        models = os.listdir(f'c2_electr/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)
        #
        # calculating MAE and plotting test and pred_test
        mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
        # mae in percent of range
        mae_range_percent[building] = mae / y_test.max() * 100

        electr_use_train_build = electr_use[building]

    metric_com_mae[regressor.upper()]=mae_range_percent

### Metrics together - RMSE

In [750]:
metric_com_rmse = pd.DataFrame(index=electr_use.columns)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    rmse_range_percent = pd.Series()
    for building in electr_use.columns:
        models = os.listdir(f'c2_electr/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)
        
        #calculating RMSE
        MSE = np.square(np.subtract(y_test, y_test_pred)).mean() 
        RMSE = math.sqrt(MSE)
        rmse_range_percent[building]=RMSE / y_test.max() * 100

    metric_com_rmse[regressor.upper()]=rmse_range_percent

### Stacked ensembling - MAE

In [ ]:
en_mae = pd.DataFrame(index=electr_use.columns) #verander naam van variabele
regressors = ['rf', 'gb', 'xg']
mae_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in electr_use.columns:
    for regressor in regressors:
        models = os.listdir(f'c2_electr/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    
    del df_d[building+"_rf"]
    del df_d[building+"_gb"]
    del df_d[building+"_xg"]
    

    mae = mean_absolute_error(y_true=df_c[building+"_en"], y_pred=df_d[building+"_en"])
    
    mae_range_percent[building] = mae / df_c[building+"_en"].max() * 100
    
    en_mae['EN']=mae_range_percent
    
print(en_mae)

### MAEs together

In [ ]:
metric_com_mae['EN']=en_mae
print(metric_com_mae)
print("\n")
print("Average MAEs")
round(metric_com_mae.mean(), 5)

### Stacked Ensembling - RMSE

In [753]:
en_rmse = pd.DataFrame(index=electr_use.columns) #verander naam van variabele
regressors = ['rf', 'gb', 'xg']
rmse_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in electr_use.columns:
    for regressor in regressors:
        models = os.listdir(f'c2_electr/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    
    del df_d[building+"_rf"]
    del df_d[building+"_gb"]
    del df_d[building+"_xg"]

    MSE = np.square(np.subtract(df_c[building+"_en"], df_d[building+"_en"])).mean() 
    RMSE = math.sqrt(MSE)
    rmse_range_percent[building]=RMSE / df_c[building+"_en"].max() * 100
    
    en_rmse['EN']=rmse_range_percent
    

### RMSEs together

In [ ]:
metric_com_rmse['EN']=en_rmse
print(metric_com_rmse)
print("\n")
print("Average RMSEs")
round(metric_com_rmse.mean(), 5)

# All models

### Adding ensemble predictions

### Deeper comparison of all 3 models

In [756]:
electr_use_pred = pd.DataFrame(index=features.index)
for m in ['rf','gb','xg']:
    for building in electr_use.columns:
        # select weather data for prediction
        city = cities[building]
        knmi_station = knmi_station_lookup[city]
        knmi_weather = weather[knmi_station]
        # preapare features for prediction
        features = pd.DataFrame()
        features['SQ'] = knmi_weather['SQ'].resample('d').mean()
        features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
        features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
        features['dow'] = features.index.dayofweek
        
        #features['month'] = features.index.month
        X_pred = features
        # do the predictions
        models = os.listdir(f'c2_electr/data/trained_models/{m}')
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/{m}/trained_{building}.joblib')
        y_pred = grid_search.predict(X_pred)
        electr_use_pred[building+'_'+m] = y_pred

In [757]:
#ensemble stacking, equal weight
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3

In [758]:
## get electr use data
engine = Engine(project=project, isAsync=False)
time_res = 'h'
electr_use_meas = pd.DataFrame(index=pd.date_range(start=train_periods['start'].min(),
                                              end=knmi_weather.index[-1],
                                              freq=time_res))
# pulls data from sp-ql
electr_builds = ['G*****', 'H8*****', 'K*****', 'V*****', 'H4****', 'W*****']
for building in [b for b in build_names if b in electr_builds]:
    start = train_periods['start'].min()
    end = knmi_weather.index[-1]
    electr_ean = electr_eans[building]
    if isinstance(electr_ean, str):
        electr = engine.select(electr_ean).group_by(time_res).mean().after(start).before(end).all() #the result is now kW power in hourly resolution, which is numerically identical to the hourly kWh value
        if len(electr) > 0:
            electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
            electr_use_meas = electr_use_meas.join(electr)
    else:
        for ean in electr_ean:
            electr = engine.select(ean).group_by(time_res).mean().after(start).before(end).all()
            if len(electr) > 0:
                electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
                electr_use_meas = electr_use_meas.join(electr)

### Cleaning

In [759]:
electr_use_meas = electr_use_meas.resample('d').sum()
filter_col = [col for col in electr_use_meas if col.startswith('WEND')] #filters the buildings that start with input name
electr_use_meas[filter_col]=electr_use_meas[filter_col].fillna(0) #adds 0's for NaN's for summation of electricity usage
electr_use_meas['W*****']=electr_use_meas[filter_col].sum(axis=1) #summation of all buildings with name filter
electr_use_meas['W*****']=electr_use_meas['W*****'].replace({'0':np.nan, 0:np.nan}) #replacing the 0's back for NaN's
electr_use_meas=electr_use_meas.drop(['W*****', 'W*****'], axis=1)
electr_use_meas.rename(columns = {'G*****':'G*****', 'H8*****':'H8*****', 'K*****':'K*****', 
                                     'V*****':'V*****', 'H4*****':'H4*****', 'T*****':'T*****',
                             'W*****':'W*****'}, inplace = True)

In [760]:
for m in ['gb','xg']:
    for building in electr_use.columns:
        electr_use_meas[building+"_"+m]=electr_use_meas[building+"_rf"]

In [762]:
month_savings = {}
month_savings_m3 = {}
total_savings_perc = pd.Series()
total_savings_m3 = pd.Series()

for m in ['rf','gb','xg']:
    for building in electr_use_meas.columns:
        size = len(building)
        start = train_periods.loc[building[:size-3], 'start']
        end = knmi_weather.index[-1]
        soc = socs[building[:size-3]]
        electr_use_meas_m = electr_use_meas.loc[start:end, building].resample('d').sum()
        electr_use_pred_m = electr_use_pred.loc[start:end, building].resample('d').sum()
        month_saving = (electr_use_pred_m - electr_use_meas_m) / electr_use_pred_m * 100
        month_savings[building] = month_saving
        month_savings_m3[building] = electr_use_pred_m - electr_use_meas_m
        total_savings_m3[building] = (electr_use_pred.loc[soc:, building] - electr_use_meas.loc[soc:, building]).sum()
        total_savings_perc[building] = total_savings_m3[building] / electr_use_pred.loc[soc:, building].sum() * 100

train_pred_mae = pd.Series()
for m in ['rf','gb','xg']:
    for building in electr_use_meas.columns:
        size = len(building)
        soc = socs[building[:size-3]]
        start = train_periods.loc[building[:size-3], 'start']
        end = train_periods.loc[building[:size-3], 'end'] #knmi_weather.index[-1]
        electr_use_meas_m = electr_use_meas.loc[start:end, building].resample('d').sum()
        electr_use_pred_m = electr_use_pred.loc[start:end, building].resample('d').sum()
        electr_use_month_mae = mean_absolute_error(y_true=electr_use_meas_m, y_pred=electr_use_pred_m)
        train_pred_mae[building] = electr_use_month_mae

In [ ]:
for building in electr_use.columns:
    t=pd.DataFrame()
    fig, ax = plt.subplots(figsize=(25, 8))
    electr_use_pred[building+"_rf"].plot(ax=ax, color='green')
    electr_use_pred[building+"_gb"].plot(ax=ax, color='blue')
    electr_use_pred[building+"_xg"].plot(ax=ax, color='red')
    electr_use_pred[building+"_en"].plot(ax=ax, color='black')
    t[building+"_measured"]=electr_use_meas[building+"_rf"]
    t.plot(ax=ax, color='gold')
    #electr_use_pred_en[building].plot(ax=ax, color='black')
    ax.set_xlim(left=electr_use_train[building].dropna().index[0])
    ax.set_title(building)
    ax.set_ylabel('Electricity use (kWh)')
    ax.legend()
    ax.axvline(x=socs[building], color='k', ls='--')
    plt.show(block=False)

### Comparison of individual models

In [847]:
electr_use_pred = pd.DataFrame(index=features.index)
for building in electr_use.columns:
    # select weather data for prediction
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    # preapare features for prediction
    features = pd.DataFrame()
    features['SQ'] = knmi_weather['SQ'].resample('d').mean()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek

    #features['month'] = features.index.month
    X_pred = features
    # do the predictions
    models = os.listdir(f'c2_electr/data/trained_models/xg')
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_electr/data/trained_models/xg/trained_{building}.joblib')
    y_pred = grid_search.predict(X_pred)
    electr_use_pred[building] = y_pred

In [849]:
## get electr use data
engine = Engine(project=project, isAsync=False)
time_res = 'h'
electr_use_meas = pd.DataFrame(index=pd.date_range(start=train_periods['start'].min(),
                                              end=knmi_weather.index[-1],
                                              freq=time_res))
# pulls data from sp-ql
electr_builds = ['G*****', 'H8*****', 'K*****', 'V*****', 'H4*****', 'W*****']
for building in [b for b in build_names if b in electr_builds]:
    start = train_periods['start'].min()
    end = knmi_weather.index[-1]
    electr_ean = electr_eans[building]
    if isinstance(electr_ean, str):
        electr = engine.select(electr_ean).group_by(time_res).mean().after(start).before(end).all() #the result is now kW power in hourly resolution, which is numerically identical to the hourly kWh value
        if len(electr) > 0:
            electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
            electr_use_meas = electr_use_meas.join(electr)
    else:
        for ean in electr_ean:
            electr = engine.select(ean).group_by(time_res).mean().after(start).before(end).all()
            if len(electr) > 0:
                electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
                electr_use_meas = electr_use_meas.join(electr)

In [850]:
electr_use_meas = electr_use_meas.resample('d').sum()
filter_col = [col for col in electr_use_meas if col.startswith('W***')] #filters the buildings that start with input name
electr_use_meas[filter_col]=electr_use_meas[filter_col].fillna(0) #adds 0's for NaN's for summation of electricity usage
electr_use_meas['W*****']=electr_use_meas[filter_col].sum(axis=1) #summation of all buildings with name filter
electr_use_meas['W*****']=electr_use_meas['W*****'].replace({'0':np.nan, 0:np.nan}) #replacing the 0's back for NaN's

In [851]:
electr_use_meas=electr_use_meas.drop(['W*****', 'W*****'], axis=1)

In [852]:
electr_use_meas.rename(columns = {'G*****':'G*****', 'H*****':'H*****', 'K*****':'K*****', 
                                     'V*****':'V*****', 'H*****':'H*****', 'T*****':'T*****',
                             'W*****':'W*****'}, inplace = True)

In [854]:
electr_use_meas.to_csv(r'electr_use_meas.csv')

In [855]:
month_savings = {}
month_savings_m3 = {}
total_savings_perc = pd.Series()
total_savings_m3 = pd.Series()
for building in electr_use_meas.columns:
    start = train_periods.loc[building, 'start']
    end = knmi_weather.index[-1]
    soc = socs[building]
    electr_use_meas_m = electr_use_meas.loc[start:end, building].resample('m').sum()
    electr_use_pred_m = electr_use_pred.loc[start:end, building].resample('m').sum()
    month_saving = (electr_use_pred_m - electr_use_meas_m) / electr_use_pred_m * 100
    month_savings[building] = month_saving
    month_savings_m3[building] = electr_use_pred_m - electr_use_meas_m
    total_savings_m3[building] = (electr_use_pred.loc[soc:, building] - electr_use_meas.loc[soc:, building]).sum()
    total_savings_perc[building] = total_savings_m3[building] / electr_use_pred.loc[soc:, building].sum() * 100

train_pred_mae = pd.Series()
for building in electr_use_meas.columns:
    soc = socs[building]
    start = train_periods.loc[building, 'start']
    end = train_periods.loc[building, 'end'] #knmi_weather.index[-1]
    electr_use_meas_m = electr_use_meas.loc[start:end, building].resample('m').sum()
    electr_use_pred_m = electr_use_pred.loc[start:end, building].resample('m').sum()
    electr_use_month_mae = mean_absolute_error(y_true=electr_use_meas_m, y_pred=electr_use_pred_m)
    train_pred_mae[building] = electr_use_month_mae

In [ ]:
for building in electr_use_meas.columns:
    total_savings_m3[building] = (electr_use_pred.loc[soc:, building] - electr_use_meas.loc[soc:, building]).sum()
    total_savings_perc[building] = ((electr_use_pred.loc[soc:, building] - electr_use_meas.loc[soc:, building]).sum() / electr_use_pred.loc[soc:, building].sum()) * 100
total_savings_perc

In [876]:
from datetime import date
from dateutil.relativedelta import relativedelta

2021-01-11 00:00:00


Timestamp('2020-07-11 00:00:00')

In [ ]:
for building in electr_use.columns:
    start = train_periods.loc[building, 'start']
    end = knmi_weather.index[-1]
    df_compare = pd.DataFrame(index=pd.date_range(start=start,
                                                     end=end,
                                                     freq=time_res))
    df_compare = df_compare.join(electr_use_meas[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    df_compare = df_compare.join(electr_use_pred[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_pred'})
    #
    df_compare = df_compare.resample('m').sum()
    soc = socs[building]#.date()
    
    # print(df_compare[building+"_meas"].loc[soc-relativedelta(months=+12):soc].mean()) ##new, could be revisitied in the future

    soc_pos = calc_soc_pos_on_plot(df_compare_monthly=df_compare, soc=soc)
    train_end_pos = calc_soc_pos_on_plot(df_compare_monthly=df_compare, soc=train_periods.loc[building, 'end'])
    #
    fig, ax = plt.subplots(figsize=(25, 8))
    err = pd.DataFrame(index=df_compare.index, data={fr'{building}_pred':train_pred_mae[building]})
    err.iloc[:int(soc_pos) + 1] = np.nan
    df_compare.plot.bar(ax=ax, color=['gold', 'darkgray'], yerr=err,
                        error_kw=dict(ecolor='darkred', lw=1, capsize=2, capthick=1))
    #soc_mark = pd.Series(index=df_compare.index, data=np.nan)
    ax.set_xticklabels([x.strftime("%Y-%m") for x in df_compare.index], rotation=45)
    plt.vlines(x=train_end_pos, ymin=0, ymax=df_compare.max().max(), color='darkred', ls='--')
    plt.vlines(x=soc_pos, ymin=0, ymax=df_compare.max().max(), color='k', ls='--')
    patches = ax.patches
    patches_m = patches[0:int(len(patches)/2)]
    patches_p = patches[int(len(patches)/2):]
    for i, p in enumerate(patches[0:int(len(patches)/2)]):
        if i > soc_pos:
            saving = int(month_savings[building].iloc[i].round(0))
            saving_m3 = int(month_savings_m3[building].iloc[i].round(0))
            width = p.get_width()
            height = max(p.get_height(), patches_p[i].get_height())
            x, y = p.get_xy()
            ax.annotate(f'{saving}%\n{saving_m3}m3', (x + width/2, y + height*1.01), ha='center')
    #
    plt.ylabel('Monthly electricity use [kWh]')
    plt.title(building)
    plt.show(block=False)

In [797]:
# calculate monthly portfolio savings
electr_use_pred_sav = electr_use_pred
electr_use_meas_sav = electr_use_meas
for building in electr_use.columns:
    electr_use_pred_sav.loc[electr_use_pred_sav.index[0]:socs[building], building] = np.nan
    electr_use_meas_sav.loc[electr_use_meas_sav.index[0]:socs[building], building] = np.nan
electr_use_sav = electr_use_pred_sav - electr_use_meas_sav
electr_use_sav = electr_use_sav.resample('m').sum()
electr_use_sav = electr_use_sav.loc[socs[electr_eans.index].min():]

In [ ]:
for building in electr_use_sav.columns:
    print(building, "total savings since SOC (kWh):", electr_use_sav[building].sum())

In [ ]:
round(electr_use_sav.sum(),2)

In [ ]:
round(electr_use_sav.sum().sum(),2)

In [ ]:
round(electr_use_sav.sum().sum()*0.23,2)

### Monthly portfolio savings

In [780]:
electr_use_pred_sav = electr_use_pred
electr_use_meas_sav = electr_use_meas
for building in electr_use.columns:
    electr_use_pred_sav.loc[electr_use_pred_sav.index[0]:socs[building], building] = np.nan
    electr_use_meas_sav.loc[electr_use_meas_sav.index[0]:socs[building], building] = np.nan
electr_use_sav = electr_use_pred_sav - electr_use_meas_sav
electr_use_sav = electr_use_sav.resample('m').sum()
electr_use_sav = electr_use_sav.loc[socs[electr_use.columns].min():]
electr_use_sav['m3_cost']=0.76 #assuming cost of m3 is 0.76 cents, could be changed
for col in electr_use_sav.columns:
    electr_use_sav[col]=round(electr_use_sav['m3_cost']*electr_use_sav[col],2)
del electr_use_sav['m3_cost']

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(20, 10))
plt.subplots_adjust(hspace=0.3)
for col, ax in zip(sorted(electr_use_sav.columns), axs.ravel()):
    index=electr_use_sav[col].index.get_loc(electr_use_sav[col].ne(0).idxmax())
    electr_use_sav[[col]].iloc[index:,].plot(ax=ax, color='gold', linewidth=3) #excludes April for now, too early in the month to draw results
    ax.set_title(col.upper())
    ax.get_legend().remove()
    ax.set_ylabel("Saving EUR")

# Forecasting

In [205]:
# forecasted features variables are read in which are daily averages from histrorical values (10 years of data)

In [ ]:
#forecast periods are defined, which is - per building - SOC up until 31st of March 2022
forecast_periods=pd.DataFrame()
forecast_periods['start'] = socs[electr_use.columns]
forecast_periods['end'] = pd.Timestamp('2022-03-31')
forecast_periods=forecast_periods[~forecast_periods.index.duplicated(keep='first')]
forecast_periods

In [715]:
engine = Engine(project=project, isAsync=False)
time_res = 'h'
electr_use = pd.DataFrame(index=pd.date_range(start=forecast_periods['start'].min(),
                                              end=forecast_periods['end'].max(),
                                              freq=time_res))

# pulls data from sp-ql
electr_builds = ['G*****', 'H*****', 'K*****', 'V*****', 'H*****', 'W*****']
for building in [b for b in build_names if b in electr_builds]:
#skips buildings with no electricity EAN data
    start = forecast_periods.loc[building, 'start']
    end = forecast_periods.loc[building, 'end']
    electr_ean = electr_eans[building]
    if isinstance(electr_ean, str):
        electr = engine.select(electr_ean).group_by(time_res).mean().after(start).before(end).all() #the result is now kW power in hourly resolution, which is numerically identical to the hourly kWh value
        if len(electr) > 0:
            electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
            electr_use = electr_use.join(electr)
        #else:
            #print(fr'{building} - electricity EAN:{electr_eans[building]} not found')
    else:
        for ean in electr_ean:
            electr = engine.select(ean).group_by(time_res).mean().after(start).before(end).all()
            if len(electr) > 0:
                electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
                electr_use = electr_use.join(electr)
            #else:
                #print(fr'{building} - electricity EAN:{ean} not found')
electr_use = electr_use.resample('d').sum() #sums the kWh to daily readings
filter_col = [col for col in electr_use if col.startswith('WEND')] #filters the buildings that start with input name
electr_use[filter_col]=electr_use[filter_col].fillna(0) #adds 0's for NaN's for summation of electricity usage
electr_use['W*****']=electr_use[filter_col].sum(axis=1) #summation of all buildings with name filter
electr_use['W*****']=electr_use['W*****'].replace({'0':np.nan, 0:np.nan}) #replacing the 0's back for NaN's
electr_use=electr_use.drop(['W*****', 'W*****'], axis=1)
electr_use.rename(columns = {'G*****':'G*****', 'H*****':'H*****', 'K*****':'K*****', 
                                     'V*****':'V*****', 'H*****':'H*****', 'T*****':'T*****',
                             'W*****':'W*****'}, inplace = True)

### Random Forest

In [ ]:
for building in electr_use.columns:
    print(fr'Training building {building}, {electr_use.columns.get_loc(building) + 1}/{len(electr_use.columns)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']
    
    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)
        
    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    features['dow'] = features.index.dayofweek
    electr_use_train_build = electr_use[building]
    
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = electr_use_train_build[(electr_use_train_build.index >= start_train) &
                            (electr_use_train_build.index < end_train)]
    #
    # checking if there is any nan in the data
    if X.isna().mean().mean() + y.isna().mean() > 0:
        raise ValueError('NaN in training data:', building)
    #
    # checking on dependency of taget on features to get a feel
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    param_grid = {'max_depth': [2, 5, 10, 20, 50, 100],
                  'min_samples_leaf': [1, 5, 10],
                  'n_estimators': [500]}
    
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                              cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    #
    # save the trained RF
    joblib.dump(grid_search.best_estimator_, fr'c2_electr/data/trained_models/f_rf/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_electr/data/trained_models/f_rf/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_electr/data/trained_models/f_rf/y_test_{building}.joblib')

### Gradient Boosting

In [ ]:
for building in electr_use.columns:
    print(fr'Training building {building}, {electr_use.columns.get_loc(building) + 1}/{len(electr_use.columns)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']
    
    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)
        
    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    features['dow'] = features.index.dayofweek
    electr_use_train_build = electr_use[building]
    
    #X = features.loc['2019-03-15':'2022-03-31']
    X = features[(features.index >= start_train) & (features.index < end_train)]


    y = electr_use_train_build[(electr_use_train_build.index >= start_train) &
                            (electr_use_train_build.index < end_train)]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


    param_grid = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10],
    }
    
    rf = GradientBoostingRegressor()
    
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                        cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    
    joblib.dump(grid_search.best_estimator_, fr'c2_electr/data/trained_models/f_gb/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_electr/data/trained_models/f_gb/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_electr/data/trained_models/f_gb/y_test_{building}.joblib')

### XGBoosting

In [ ]:
for building in electr_use.columns:
    print(fr'Training building {building}, {electr_use.columns.get_loc(building) + 1}/{len(electr_use.columns)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']
    
    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)
        
    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    features['dow'] = features.index.dayofweek
    electr_use_train_build = electr_use[building]
    
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = electr_use_train_build[(electr_use_train_build.index >= start_train) &
                            (electr_use_train_build.index < end_train)]
    if X.isna().mean().mean() + y.isna().mean() > 0:
        raise ValueError('NaN in training data:', building)
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    param_grid={
                "learning_rate": (0.05, 0.10, 0.15, 0.2),
                "max_depth": [3, 4, 5, 6, 8],
                "min_child_weight": [1, 3, 5, 7]
                }

    rf = XGBRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                              cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)

    grid_search.fit(X_train, y_train)

    joblib.dump(grid_search.best_estimator_, fr'c2_electr/data/trained_models/f_xg/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_electr/data/trained_models/f_xg/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_electr/data/trained_models/f_xg/y_test_{building}.joblib')

### Predictions

In [833]:
warnings.filterwarnings("ignore")
electr_use_pred = pd.DataFrame(index=features.index)
for m in ['rf','gb','xg']:
    for building in electr_use.columns:
        # select weather data for prediction

        city = cities[building]
        knmi_station = knmi_station_lookup[city]
        knmi_weather = weather[knmi_station]

        # preapare features for prediction

        if knmi_station==240:
            features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
        if knmi_station==344:
            features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
        if knmi_station==370:
            features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
        if knmi_station==260:
            features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
        if knmi_station==275:
            features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)

        features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')

        features['dow'] = features.index.dayofweek

        electr_use_train_build = electr_use[building]
        X_pred = features #using KNMI weather data 2011-2022/04 and the trained model, predictions are made on energy consumption
        models = os.listdir(f'c2_electr/data/trained_models/f_{m}')
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/f_{m}/trained_{building}.joblib')
        y_pred = grid_search.predict(X_pred)
        electr_use_pred[building+'_'+m] = y_pred #gives energy consumption for model for building 2011-2022
        #print(electr_use_pred)

In [834]:
#ensemble stacking, equal weight
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3
electr_use_pred['*****_en']=(electr_use_pred['*****_rf']+electr_use_pred['*****_gb']+electr_use_pred['*****_xg'])/3

### Actual

In [835]:
## get electr use data
engine = Engine(project=project, isAsync=False)
time_res = 'h'
electr_use_meas = pd.DataFrame(index=pd.date_range(start=forecast_periods['start'].min(),
                                              end=knmi_weather.index[-49]+dt.timedelta(30),
                                              freq=time_res))
# pulls data from sp-ql
electr_builds = ['G*****', 'H*****', 'K*****', 'V*****', 'H*****', 'W*****']
for building in [b for b in build_names if b in electr_builds]:
    start = forecast_periods['start'].min()
    end = knmi_weather.index[-1]+dt.timedelta(28)
    electr_ean = electr_eans[building]
    if isinstance(electr_ean, str):
        electr = engine.select(electr_ean).group_by(time_res).mean().after(start).before(end).all() #the result is now kW power in hourly resolution, which is numerically identical to the hourly kWh value
        if len(electr) > 0:
            electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
            electr_use_meas = electr_use_meas.join(electr)
    else:
        for ean in electr_ean:
            electr = engine.select(ean).group_by(time_res).mean().after(start).before(end).all()
            if len(electr) > 0:
                electr = electr.rename(columns={electr.columns[0]:fr'{building}_{electr.columns[0]}'})
                electr_use_meas = electr_use_meas.join(electr)
                
electr_use_meas = electr_use_meas.resample('d').sum()
filter_col = [col for col in electr_use_meas if col.startswith('W***')] #filters the buildings that start with input name
electr_use_meas[filter_col]=electr_use_meas[filter_col].fillna(0) #adds 0's for NaN's for summation of electricity usage
electr_use_meas['W****']=electr_use_meas[filter_col].sum(axis=1) #summation of all buildings with name filter
electr_use_meas['W****']=electr_use_meas['W*****'].replace({'0':np.nan, 0:np.nan}) #replacing the 0's back for NaN's
electr_use_meas=electr_use_meas.drop(['W*****', 'W*****'], axis=1)
electr_use_meas.rename(columns = {'G*****':'G*****', 'H*****':'H*****', 'K*****':'K*****', 
                                     'V*****':'V*****', 'H*****':'H*****', 'T*****':'T*****',
                             'W*****':'W*****'}, inplace = True)

In [836]:
def calc_date_pos_on_plot(df_compare_monthly):
    soc=pd.Timestamp('2022-04-01 00:00:00')
    sm = list(df_compare_monthly.index < soc).index(False) # first False is start month
    return sm - 0.5 # place of dashed line on plot in 0-1 x coord range

In [ ]:
for building in electr_use.columns:
    start = forecast_periods.loc[building, 'start']
    end = electr_use_pred.index[-1]
    df_compare = pd.DataFrame(index=pd.date_range(start=start,
                                                     end=end,
                                                     freq=time_res))
    df_compare = df_compare.join(electr_use_meas[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    df_compare = df_compare.join(electr_use_pred[[building+"_rf"]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    #
    df_compare = df_compare.resample('m').sum()

    soc = socs[building]#.date()
    soc_pos = calc_date_pos_on_plot(df_compare_monthly=df_compare)
    train_end_pos = calc_date_pos_on_plot(df_compare_monthly=df_compare)
    #
    fig, ax = plt.subplots(figsize=(25, 8))
    df_compare.plot.bar(ax=ax, color=['gold', 'darkgray'], yerr=err,
                        error_kw=dict(ecolor='darkred', lw=1, capsize=2, capthick=1))
    #soc_mark = pd.Series(index=df_compare.index, data=np.nan)
    ax.set_xticklabels([x.strftime("%Y-%m") for x in df_compare.index], rotation=45)
    plt.vlines(x=train_end_pos, ymin=0, ymax=df_compare.max().max(), color='darkred', ls='--')
    plt.vlines(x=soc_pos, ymin=0, ymax=df_compare.max().max(), color='k', ls='--')
    patches = ax.patches
    patches_m = patches[0:int(len(patches)/2)]
    patches_p = patches[int(len(patches)/2):]

    plt.ylabel('Monthly electricity use [kWh]')
    plt.title(building)
    plt.show(block=False)

In [ ]:
for building in electr_use.columns:
    start = forecast_periods.loc[building, 'start']
    end = electr_use_pred.index[-1]
    df_compare = pd.DataFrame(index=pd.date_range(start=start,
                                                     end=end,
                                                     freq=time_res))
    df_compare = df_compare.join(electr_use_meas[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    df_compare = df_compare.join(electr_use_pred[[building+"_rf"]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    #
    df_compare = df_compare.resample('m').sum()

    soc = socs[building]#.date()
    soc_pos = calc_date_pos_on_plot(df_compare_monthly=df_compare)
    train_end_pos = calc_date_pos_on_plot(df_compare_monthly=df_compare)
    #
    fig, ax = plt.subplots(figsize=(25, 8))
    df_compare.plot.bar(ax=ax, color=['gold', 'darkgray'], yerr=err,
                        error_kw=dict(ecolor='darkred', lw=1, capsize=2, capthick=1))
    #soc_mark = pd.Series(index=df_compare.index, data=np.nan)
    ax.set_xticklabels([x.strftime("%Y-%m") for x in df_compare.index], rotation=45)
    plt.vlines(x=train_end_pos, ymin=0, ymax=df_compare.max().max(), color='darkred', ls='--')
    plt.vlines(x=soc_pos, ymin=0, ymax=df_compare.max().max(), color='k', ls='--')
    patches = ax.patches
    patches_m = patches[0:int(len(patches)/2)]
    patches_p = patches[int(len(patches)/2):]

    plt.ylabel('Monthly electricity use [kWh]')
    plt.title(building)
    plt.show(block=False)

In [738]:
metric_com_mae = pd.DataFrame(index=electr_use.columns)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    mae_range_percent = pd.Series()
    for building in electr_use.columns:
        models = os.listdir(f'c2_electr/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)

        #
        # calculating MAE and plotting test and pred_test
        mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
        # mae in percent of range
        mae_range_percent[building] = mae / y_test.max() * 100

        electr_use_train_build = electr_use[building]

    metric_com_mae[regressor.upper()]=mae_range_percent

In [739]:
metric_com_rmse = pd.DataFrame(index=electr_use.columns)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    rmse_range_percent = pd.Series()
    for building in electr_use.columns:
        models = os.listdir(f'c2_electr/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)
        
        #calculating RMSE
        MSE = np.square(np.subtract(y_test, y_test_pred)).mean() 
        RMSE = math.sqrt(MSE)
        rmse_range_percent[building]=RMSE / y_test.max() * 100

    metric_com_rmse[regressor.upper()]=rmse_range_percent

In [740]:
en_mae = pd.DataFrame(index=electr_use.columns)
regressors = ['rf', 'gb', 'xg']
mae_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in electr_use.columns:
    for regressor in regressors:
        models = os.listdir(f'c2_electr/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    
    del df_d[building+"_rf"]
    del df_d[building+"_gb"]
    del df_d[building+"_xg"]
    

    mae = mean_absolute_error(y_true=df_c[building+"_en"], y_pred=df_d[building+"_en"])
    
    mae_range_percent[building] = mae / df_c[building+"_en"].max() * 100

    en_mae['EN']=mae_range_percent

### MAEs

In [ ]:
metric_com_mae['EN']=en_mae
print(metric_com_mae)
print("\n")
print("Average MAEs")
round(metric_com_mae.mean(), 5)

In [742]:
en_rmse = pd.DataFrame(index=electr_use.columns) #verander naam van variabele
regressors = ['rf', 'gb', 'xg']
rmse_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in electr_use.columns:
    for regressor in regressors:
        models = os.listdir(f'c2_electr/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_electr/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    
    del df_d[building+"_rf"]
    del df_d[building+"_gb"]
    del df_d[building+"_xg"]

    MSE = np.square(np.subtract(df_c[building+"_en"], df_d[building+"_en"])).mean() 
    RMSE = math.sqrt(MSE)
    rmse_range_percent[building]=RMSE / df_c[building+"_en"].max() * 100
    
    en_rmse['EN']=rmse_range_percent

### RMSEs

In [ ]:
metric_com_rmse['EN']=en_rmse
print(metric_com_rmse)
print("\n")
print("Average RMSEs")
round(metric_com_rmse.mean(), 5)

### Individual Predictions

In [885]:
warnings.filterwarnings("ignore")
electr_use_pred = pd.DataFrame(index=features.index)
for building in electr_use.columns:
    
    # select weather data for prediction

    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]

    # preapare features for prediction

    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)

    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')

    features['dow'] = features.index.dayofweek

    electr_use_train_build = electr_use[building]
    X_pred = features #using KNMI weather data 2011-2022/04 and the trained model, predictions are made on energy consumption
    models = os.listdir(f'c2_electr/data/trained_models/f_xg')
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_electr/data/trained_models/f_xg/trained_{building}.joblib')
    y_pred = grid_search.predict(X_pred)
    electr_use_pred[building] = y_pred #gives energy consumption for model for building 2011-2022
    #print(electr_use_pred)

In [ ]:
electr_use_pred=electr_use_pred.loc['2022-04-01':,]
electr_use_pred=electr_use_pred.resample('m').sum()
electr_use_pred.mean()*0.23